# Imports

In [ ]:
import numpy as np
import scipy.linalg as spla
import scipy.io as spio
import matplotlib.pyplot as plt

from pymor.models.iosys import LTIModel

In [ ]:
plt.rcParams['figure.dpi'] = 100
plt.rcParams['axes.grid'] = True

# Build model

In [ ]:
mat = spio.loadmat('data/ABCE.mat')
mat.keys()

In [ ]:
mu = np.sqrt(10) * np.array([0.2, 0.4, 0.6, 0.8])
A = mat['A0']
for i in range(4):
    A += mu[i] * mat[f'A{i + 1}']
B = mat['B']
C = mat['C']
E = mat['E']

In [ ]:
fom = LTIModel.from_matrices(A, B, C, E=E)

In [ ]:
fom

In [ ]:
print(fom)

# Bode plot

In [ ]:
w = np.logspace(-2, 4, 100)
fig, axs = plt.subplots(2, 1, figsize=(6, 8), constrained_layout=True)
_ = fom.bode_plot(w, ax=np.array(4 * [[axs[0]], [axs[1]]]))

In [ ]:
_ = fom.mag_plot(w)

# Bitangential Hermite interpolation

Full-order model:
$$
\begin{align*}
  E \dot{x}(t) & = A x(t) + B u(t), \\
  y(t) & = C x(t).
\end{align*}
$$
Reduced-order model:
$$
\begin{align*}
  \hat{E} \dot{\hat{x}}(t) & = \hat{A} \hat{x}(t) + \hat{B} u(t), \\
  \hat{y}(t) & = \hat{C} \hat{x}(t),
\end{align*}
$$
where
$$
\begin{align*}
  \hat{E} = W^T E V, \quad
  \hat{A} = W^T A V, \quad
  \hat{B} = W^T B, \quad
  \hat{C} = C V.
\end{align*}
$$

Theorem 1 from [Antoulas/Beattie/Gugercin 2010]:
- if $(\sigma E - A)^{-1} B b \in \operatorname{im}(V)$,
then $H(\sigma) b = \hat{H}(\sigma) b$
- if $(\sigma E - A)^{-*} C^T c \in \operatorname{im}(W)$,
then $c^* H(\sigma) = c^* \hat{H}(\sigma)$
- if both,
then also $c^* H'(\sigma) b = c^* \hat{H}'(\sigma) b$.

In [ ]:
from pymor.reductors.interpolation import LTIBHIReductor

In [ ]:
sigma = [10j, -10j]
b = np.array([[1], [1]])
b = fom.B.source.from_numpy(b)
c = np.array([[1, 0, 0, 0], [1, 0, 0, 0]])
c = fom.C.range.from_numpy(c)

In [ ]:
interp = LTIBHIReductor(fom)

In [ ]:
rom_interp = interp.reduce(sigma, b, c)

In [ ]:
rom_interp

In [ ]:
print(rom_interp)

In [ ]:
fom.eval_tf(10j)

In [ ]:
rom_interp.eval_tf(10j)

In [ ]:
fom.eval_tf(10j) - rom_interp.eval_tf(10j)

In [ ]:
fom.eval_dtf(10j)

In [ ]:
fom.eval_dtf(10j) - rom_interp.eval_dtf(10j)

In [ ]:
err_interp = fom - rom_interp

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(6, 8), constrained_layout=True)
_ = err_interp.bode_plot(w, ax=np.array(4 * [[axs[0]], [axs[1]]]))

In [ ]:
_ = err_interp.mag_plot(w)

# Iterative rational Krylov algorithm

Interpolatory necessary conditions for $\mathcal{H}_2$-optimality
[Meier/Luenberger 1967, Antoulas/Beattie/Gugercin 2006/2008/2010]:
If $\hat{H}(s) = \sum_{i = 1}^r \frac{c_i b_i^*}{s - \lambda_i}$ is an
$\mathcal{H}_2$-optimal ROM for $H$,
then
$$
\begin{align*}
  H(-\overline{\lambda_i}) b_i
  & = \hat{H}(-\overline{\lambda_i}) b_i, \\
  c_i^* H(-\overline{\lambda_i})
  & = c_i^* \hat{H}(-\overline{\lambda_i}), \\
  c_i^* H'(-\overline{\lambda_i}) b_i
  & = c_i^* \hat{H}'(-\overline{\lambda_i}) b_i.
\end{align*}
$$

In [ ]:
from pymor.reductors.h2 import IRKAReductor

In [ ]:
irka = IRKAReductor(fom)

In [ ]:
rom_irka = irka.reduce(10)

In [ ]:
_ = plt.semilogy(irka.conv_crit, '.-')

In [ ]:
irka_poles = rom_irka.poles()
_ = plt.plot(irka_poles.real, irka_poles.imag, '.')

In [ ]:
err_irka = fom - rom_irka

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(6, 8), constrained_layout=True)
_ = err_irka.bode_plot(w, ax=np.array(4 * [[axs[0]], [axs[1]]]))

In [ ]:
_ = err_irka.mag_plot(w)